In [ ]:
import pandas as pd
from sympy import sympify, S, Eq, lambdify
from IPython.display import display

from scipy.special import sph_harm

import numpy as np

# Impelemtentation of operator 3-2 in numeric PWD module
The operator of choice is
\begin{equation}
    \newcommand{\va}[1]{\vec{\boldsymbol{#1}}}
    \newcommand{\p}{\va{p}}
    \newcommand{\q}{\va{q}}
    \newcommand{\kk}{\va{k}}
    \newcommand{\m}{m_{\pi}}
\end{equation}

\begin{equation}
    \label{op-def}\tag{1}
    \mathcal{O}_{3,two}
    =
	\tilde{c}_{3,two}  
	\frac{
		(\va{\sigma}_{\chi}  \cdot \kk_1)
		(  \va{\sigma}_2 \cdot \kk_1)
	}
	{
		\kk_1^2 + \m^2
	} 
	\tau_2^3
    \, , \qquad
    \kk_1 = \tfrac{\q}{2} + \p - \p'
\end{equation}

We intend to crosscheck analytic PWD with numeric PWD.
To make it more feasible to compute the analytic PWD we drop the denominator and isospin components in eq. \eqref{op-def}.

\begin{equation}
    \label{op-def-no-denom}\tag{2}
    \tilde{\mathcal{O}}_{3,two}
    =
	(\va{\sigma}_{\chi}  \cdot \kk_1)
	(  \va{\sigma}_2 \cdot \kk_1)
\end{equation}

I use

$a = \alpha$, $b_1 = \beta$, $b_2 = \beta^*$, $d = \delta$ and $e = \Omega$ where, 

and

\begin{align*}
\alpha &= q + 2 p_1 \cos \theta_1 - 2 p_2 \cos \theta_2
\\
\beta &  = e^{i\phi} p_1 \sin\theta_1 - p_2 \sin\theta_2
\\
\Omega &= 4 p_1^2 + 4 p_2^2 - 4 p_1^2 \cos ^2 \theta_1 - 4 p_2^2 \cos ^2 \theta_2
			-8 p_1 p_2 \cos\phi \sin \theta_1 \sin \theta_2
\\
\delta &= p_1^2 + p_2^2 +\tfrac{q^2}{4} + m_{\pi}^2 + p_1  q \cos \theta_1 - p_2 q \cos\theta_2
			-2 p_1 p_2 \cos\theta_1 \cos \theta_2 - 2 p_1 p_2 \cos \phi \sin \theta_1 \sin \theta_2
\end{align*}

In [ ]:
INPUT = "input-op-32.csv"

In [ ]:
df = pd.read_csv(INPUT).applymap(lambda el: sympify(el))
df.head()

In [ ]:
substitutions = {
    "a": S("q + 2 * p1 * x1 - 2*p2 *x2"),
    "b_1": S("exp(I*phi) * p1 * sqrt(1-x1**2) - p2*sqrt(1-x2**2)"),
    "b_2": S("exp(-I*phi) * p1 * sqrt(1-x1**2) - p2*sqrt(1-x2**2)"),
    "e": S(
        "4*p1**2 * (1-x1**2) + 4*p2**2 * (1-x2**2) - 8*p1*p2*cos(phi)*sqrt(1-x1**2)*sqrt(1-x2**2)"
    ),
}
for key, val in substitutions.items():
    display(Eq(S(key), val))

In [ ]:
expr = df.mat[0]
expr = expr.subs(substitutions)
expr

In [ ]:
f = lambdify(("p1", "p2", "q", "x1", "x2", "phi"), expr, modules="numpy")

In [ ]:
class SympyTensorConverter:
    def __init__(self, expr, args):
        self.expr = expr
        self.args = args
        self._func = lambdify(args, expr, modules="numpy")

    @staticmethod
    def _args_to_flat_tensor(*args):
        return [
            arr.flatten() for arr in np.meshgrid(*args, sparse=False, indexing="ij")
        ]

    @staticmethod
    def _flat_to_tensor(arr, shape):
        return arr.reshape(shape)

    def __call__(self, *args):
        shape = tuple(len(arg) for arg in args)
        flat_args = self._args_to_flat_tensor(*args)
        flat_res = self._func(*flat_args)
        return self._flat_to_tensor(flat_res, shape)

In [ ]:
f = SympyTensorConverter(expr, ("p1", "p2", "q", "x1", "x2", "phi"))

In [ ]:
p1 = np.linspace(0, 10, 10)
p2 = np.linspace(0, 10, 20)
q = np.linspace(0, 1, 2)
x1, w1 = np.polynomial.legendre.leggauss(5)
x2, w2 = np.polynomial.legendre.leggauss(15)
phi, wphi = np.linspace(0, 2 * np.pi, 6), np.ones(6) * 2 * np.pi / 6

In [ ]:
shape = (10, 20, 2, 5, 15, 6)

In [ ]:
out_sympy = f(p1, p2, q, x1, x2, phi)
out_sympy.shape

In [ ]:
expr

In [ ]:
def get_res_numpy():
    zeros = np.zeros(shape)

    pp1 = p1.reshape((10, 1, 1, 1, 1, 1)) + zeros
    pp2 = p2.reshape((1, 20, 1, 1, 1, 1)) + zeros
    qq = q.reshape((1, 1, 2, 1, 1, 1)) + zeros
    xx1 = x1.reshape((1, 1, 1, 5, 1, 1)) + zeros
    xx2 = x2.reshape((1, 1, 1, 1, 15, 1)) + zeros
    pphi = phi.reshape((1, 1, 1, 1, 1, 6)) + zeros

    beta = pp1 * np.sqrt(1 - xx1 ** 2) * np.exp(1j * pphi)
    beta -= pp2 * np.sqrt(1 - xx2 ** 2)
    alpha = 2 * pp1 * xx1 - 2 * pp2 * xx2 + qq

    return alpha * beta


out = get_res_numpy()
out.shape

In [ ]:
np.testing.assert_equal(out, out_sympy)

In [ ]:
%timeit f(p1, p2, q, x1, x2, phi)

In [ ]:
%timeit get_res_numpy()

In [ ]:
out.shape

In [ ]:
bb = np.random.normal(size=(5, 1, 1))

In [ ]:
out * bb